# PINN for Burger's Equation
Resources:
https://github.com/benmoseley/harmonic-oscillator-pinn/blob/main/Harmonic%20oscillator%20PINN.ipynb
https://www.mathworks.com/help/deeplearning/ug/solve-partial-differential-equations-using-deep-learning.html
https://colab.research.google.com/github/janblechschmidt/PDEsByNNs/blob/main/PINN_Solver.ipynb#scrollTo=pQhmdtsqZEkj

In [ ]:
import numpy as np
import torch

boundary_points = 25 # actually 2*boundary_points, so 50
# boundary conditions
x_b = np.ones(2*boundary_points)
x_b[:boundary_points] = -1*x_b[:boundary_points]
t_b = np.linspace(0, 1, boundary_points)
t_b = np.concatenate((t_b, t_b))
u_b = np.zeros(2*boundary_points)

# inital conditions (t=0) 
initial_points = 50 # initial conditions
t_0 = np.zeros(initial_points)
x_0 = np.linspace(-1, 1, initial_points)
u_0 = -np.sin(x_0*np.pi)

# combine
t_c = np.concatenate(t_b, t_0)
x_c = np.concatenate(x_b, x_0)
inp_c = np.vstack([t_c, x_c])
u_c = np.concatenate(u_b, u_0)

# collocation points
num_points = 10000
t = np.zeros(0, 1, num_points)
x = np.linspace(-1, 1, num_points)
collocation = np.vstack([t_0, x_0])

# convert to tensor
inp_c = torch.from_numpy(inp_c)
u_c = torch.from_numpy(u_c)
collocation = torch.from_numpy(collocation)

In [ ]:
import torch.nn as nn
import torch.optim as optim

def NeuralNet(nn.Module):
    def __init__(self, in_size, out_size):
        self.in_size = in_size
        self.out_size = out_size
        self.model = nn.Sequential(OrderedDict([
                                  ('linear1', nn.Linear(in_size, 50)),
                                  ('tanh1', nn.Tanh()),
                                  ('linear2', nn.Linear(50, 200)),
                                  ('tanh2', nn.Tanh()),
                                  ('linear3', nn.Linear(200, 500)),
                                  ('tanh3', nn.Tanh()),
                                  ('linear4', nn.Linear(500, out_size)),
                                  ('tanh4', nn.Tanh()),            
                                ]))
    def forward(self, x):
        return self.model(x)

In [ ]:
model = NeuralNet(2, 1)
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
import matplotlib.pyplot as plt

viscosity = 0.01/np.pi

epochs=200

losses = []
for i in range(epochs):
    # conditions loss
    u = model(inp_c)
    loss = torch.mean((u_c - u)**2) # MSE
    
    # physics based loss
    u = model(collocation)
    dt  = torch.autograd.grad(t, collocation, torch.ones_like(u), create_graph=True)[0] # computes du/dt
    dx  = torch.autograd.grad(x, collocation, torch.ones_like(u), create_graph=True)[0] # computes du/dx
    dx2 = torch.autograd.grad(dx,  collocation, torch.ones_like(dx),  create_graph=True)[0] # computes d^2u/dx^2
    loss += dt + u*dx - viscosity*dx2

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    losses.append(loss)

plt.plot(range(epochs), losses)